In [1]:
from pathlib import Path
from indigo import Indigo
import numpy as np
import pandas as pd
import asyncio

from substrucure_finder import BucketsInitializer
from substrucure_finder import SearchEngine, ThreadPoolSearchEngine

import fp_utils
from fp_utils.finders import Finder
from fp_utils.tests import FinderSpeedTester
from fp_utils.catch_time import CatchTime

In [2]:
fp_utils.settings.init_fp_utils()

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
data_path = Path("../data/")

In [4]:
zero_columns_name = '../data/zero_columns'

# Build structure

In [32]:
data_name = '17kk_100_heur2000'

In [33]:
initializer = BucketsInitializer(data_path / data_name, columns_count=100)

In [34]:
(data_path / data_name / 'buckets').mkdir(exist_ok=True)

In [35]:
%%time
initializer.init_buckets()

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Start init 75
Finish init 75
Start init 77
Finish init 77
Start init 78
Finish init 78
Start init 79
Finish init 79
Start init 81
Finish init 81
Start init 82
Finish init 82
Start init 87
Finish init 87
Start init 88
Finish init 88
Start init 89
Finish init 89
Start init 90
Finish init 90
Start init 93
Finish init 93
Start init 94
Finish init 94
Start init 95
Finish init 95
Start init 96
Finish init 96
Start init 105
Finish init 105
Start init 106
Finish init 106
Start init 107
Finish init 107
Start init 108
Finish init 108
Start init 109
Finish init 109
Start init 110
Finish init 110
Start init 113
Finish init 113
Start init 115
Finish init 115
Start init 116
Finish init 116
Start init 117
Finish init 117
Start init 118
Finish init 118
Start init 123
Finish init 123
Start init 124
Finish init 124
Start init 125
Finish init 125
Start init 126
Fin

Finish init 642
Start init 647
Finish init 647
Start init 648
Finish init 648
Start init 653
Finish init 653
Start init 654
Finish init 654
Start init 655
Finish init 655
Start init 656
Finish init 656
Start init 665
Finish init 665
Start init 666
Finish init 666
Start init 667
Finish init 667
Start init 669
Finish init 669
Start init 673
Finish init 673
Start init 674
Finish init 674
Start init 675
Finish init 675
Start init 676
Finish init 676
Start init 677
Finish init 677
Start init 678
Finish init 678
Start init 685
Finish init 685
Start init 686
Finish init 686
Start init 687
Finish init 687
Start init 688
Finish init 688
Start init 691
Finish init 691
Start init 692
Finish init 692
Start init 699
Finish init 699
Start init 700
Finish init 700
Start init 701
Finish init 701
Start init 702
Finish init 702
Start init 703
Finish init 703
Start init 704
Finish init 704
Start init 705
Finish init 705
Start init 706
Finish init 706
Start init 719
Finish init 719
Start init 720
Finish i

Finish init 1166
Start init 1167
Finish init 1167
Start init 1168
Finish init 1168
Start init 1171
Finish init 1171
Start init 1172
Finish init 1172
Start init 1173
Finish init 1173
Start init 1174
Finish init 1174
Start init 1177
Finish init 1177
Start init 1178
Finish init 1178
Start init 1181
Finish init 1181
Start init 1182
Finish init 1182
Start init 1185
Finish init 1185
Start init 1186
Finish init 1186
Start init 1187
Finish init 1187
Start init 1188
Finish init 1188
Start init 1201
Finish init 1201
Start init 1202
Finish init 1202
Start init 1205
Finish init 1205
Start init 1206
Finish init 1206
Start init 1210
Finish init 1210
Start init 1212
Finish init 1212
Start init 1213
Finish init 1213
Start init 1214
Finish init 1214
Start init 1215
Finish init 1215
Start init 1216
Finish init 1216
Start init 1219
Finish init 1219
Start init 1220
Finish init 1220
Start init 1229
Finish init 1229
Start init 1230
Finish init 1230
Start init 1231
Finish init 1231
Start init 1232
Finish ini

Finish init 1664
Start init 1665
Finish init 1665
Start init 1666
Finish init 1666
Start init 1671
Finish init 1671
Start init 1672
Finish init 1672
Start init 1673
Finish init 1673
Start init 1674
Finish init 1674
Start init 1677
Finish init 1677
Start init 1678
Finish init 1678
Start init 1679
Finish init 1679
Start init 1681
Finish init 1681
Start init 1683
Finish init 1683
Start init 1684
Finish init 1684
Start init 1686
Finish init 1686
Start init 1691
Finish init 1691
Start init 1693
Finish init 1693
Start init 1697
Finish init 1697
Start init 1698
Finish init 1698
Start init 1699
Finish init 1699
Start init 1700
Finish init 1700
Start init 1701
Finish init 1701
Start init 1702
Finish init 1702
Start init 1703
Finish init 1703
Start init 1704
Finish init 1704
Start init 1705
Finish init 1705
Start init 1706
Finish init 1706
Start init 1715
Finish init 1715
Start init 1716
Finish init 1716
Start init 1717
Finish init 1717
Start init 1718
Finish init 1718
Start init 1719
Finish ini

Finish init 2163
Start init 2164
Finish init 2164
Start init 2165
Finish init 2165
Start init 2166
Finish init 2166
Start init 2171
Finish init 2171
Start init 2172
Finish init 2172
Start init 2173
Finish init 2173
Start init 2174
Finish init 2174
Start init 2175
Finish init 2175
Start init 2176
Finish init 2176
Start init 2177
Finish init 2177
Start init 2182
Finish init 2182
Start init 2193
Finish init 2193
Start init 2194
Finish init 2194
Start init 2199
Finish init 2199
Start init 2200
Finish init 2200
Start init 2201
Finish init 2201
Start init 2202
Finish init 2202
Start init 2203
Finish init 2203
Start init 2204
Finish init 2204
Start init 2207
Finish init 2207
Start init 2208
Finish init 2208
Start init 2209
Finish init 2209
Start init 2210
Finish init 2210
Start init 2211
Finish init 2211
Start init 2212
Finish init 2212
Start init 2215
Finish init 2215
Start init 2216
Finish init 2216
Start init 2221
Finish init 2221
Start init 2222
Finish init 2222
Start init 2227
Finish ini

...

Finish init 63697
Start init 63698
Finish init 63698
Start init 63699
Finish init 63699
Start init 63700
Finish init 63700
Start init 63703
Finish init 63703
Start init 63704
Finish init 63704
Start init 63705
Finish init 63705
Start init 63706
Finish init 63706
Start init 63707
Finish init 63707
Start init 63708
Finish init 63708
Start init 63709
Finish init 63709
Start init 63710
Finish init 63710
Start init 63715
Finish init 63715
Start init 63716
Finish init 63716
Start init 63719
Finish init 63719
Start init 63720
Finish init 63720
Start init 63721
Finish init 63721
Start init 63722
Finish init 63722
Start init 63727
Finish init 63727
Start init 63728
Finish init 63728
Start init 63737
Finish init 63737
Start init 63738
Finish init 63738
Start init 63739
Finish init 63739
Start init 63740
Finish init 63740
Start init 63745
Finish init 63745
Start init 63746
Finish init 63746
Start init 63747
Finish init 63747
Start init 63748
Finish init 63748
Start init 63749
Finish init 63749
St

Finish init 64158
Start init 64159
Finish init 64159
Start init 64160
Finish init 64160
Start init 64161
Finish init 64161
Start init 64162
Finish init 64162
Start init 64163
Finish init 64163
Start init 64164
Finish init 64164
Start init 64165
Finish init 64165
Start init 64166
Finish init 64166
Start init 64175
Finish init 64175
Start init 64176
Finish init 64176
Start init 64177
Finish init 64177
Start init 64178
Finish init 64178
Start init 64179
Finish init 64179
Start init 64180
Finish init 64180
Start init 64181
Finish init 64181
Start init 64182
Finish init 64182
Start init 64189
Finish init 64189
Start init 64190
Finish init 64190
Start init 64193
Finish init 64193
Start init 64194
Finish init 64194
Start init 64195
Finish init 64195
Start init 64196
Finish init 64196
Start init 64197
Finish init 64197
Start init 64198
Finish init 64198
Start init 64201
Finish init 64201
Start init 64202
Finish init 64202
Start init 64207
Finish init 64207
Start init 64208
Finish init 64208
St

Finish init 64611
Start init 64612
Finish init 64612
Start init 64613
Finish init 64613
Start init 64614
Finish init 64614
Start init 64615
Finish init 64615
Start init 64616
Finish init 64616
Start init 64623
Finish init 64623
Start init 64624
Finish init 64624
Start init 64625
Finish init 64625
Start init 64626
Finish init 64626
Start init 64627
Finish init 64627
Start init 64628
Finish init 64628
Start init 64629
Finish init 64629
Start init 64630
Finish init 64630
Start init 64637
Finish init 64637
Start init 64638
Finish init 64638
Start init 64641
Finish init 64641
Start init 64642
Finish init 64642
Start init 64647
Finish init 64647
Start init 64648
Finish init 64648
Start init 64649
Finish init 64649
Start init 64650
Finish init 64650
Start init 64651
Finish init 64651
Start init 64652
Finish init 64652
Start init 64653
Finish init 64653
Start init 64654
Finish init 64654
Start init 64663
Finish init 64663
Start init 64664
Finish init 64664
Start init 64665
Finish init 64665
St

Finish init 65066
Start init 65067
Finish init 65067
Start init 65068
Finish init 65068
Start init 65071
Finish init 65071
Start init 65072
Finish init 65072
Start init 65075
Finish init 65075
Start init 65076
Finish init 65076
Start init 65081
Finish init 65081
Start init 65082
Finish init 65082
Start init 65085
Finish init 65085
Start init 65086
Finish init 65086
Start init 65089
Finish init 65089
Start init 65090
Finish init 65090
Start init 65091
Finish init 65091
Start init 65092
Finish init 65092
Start init 65093
Finish init 65093
Start init 65094
Finish init 65094
Start init 65095
Finish init 65095
Start init 65096
Finish init 65096
Start init 65101
Finish init 65101
Start init 65102
Finish init 65102
Start init 65103
Finish init 65103
Start init 65104
Finish init 65104
Start init 65115
Finish init 65115
Start init 65116
Finish init 65116
Start init 65117
Finish init 65117
Start init 65118
Finish init 65118
Start init 65123
Finish init 65123
Start init 65124
Finish init 65124
St

Finish init 65523
Start init 65524
Finish init 65524
Start init 65525
Finish init 65525
Start init 65526
Finish init 65526
Start init 65531
Finish init 65531
Start init 65532
Finish init 65532
Start init 65533
Finish init 65533
Start init 65534
Finish init 65534
Start init 65537
Finish init 65537
Start init 65538
Finish init 65538
Start init 65539
Finish init 65539
Start init 65540
Finish init 65540
CPU times: user 1h 50min 30s, sys: 4min 22s, total: 1h 54min 52s
Wall time: 1h 56min 50s


# Search Queries

In [9]:
with open(zero_columns_name, 'r') as f:
    zero_columns = list(map(int, f.read().split()))
assert list(sorted(zero_columns)) == zero_columns

In [10]:
full_columns = list(sorted(set(range(3736)) - set(zero_columns)))

In [11]:
def bin_format(number, num_of_bits=3736):
    fp = list(map(int, list(bin(int(number, 16))[2:].zfill(num_of_bits))))
    return np.fromiter(fp, dtype=int)

In [12]:
def smiles_to_fingerprint(smiles):
    indigo_mol_to_test = Indigo().loadMolecule(smiles)
    full_fp = bin_format(indigo_mol_to_test.fingerprint("sub").toString())
    return np.fromiter(pd.Series(full_fp)[full_columns].values, dtype=bool)

In [13]:
@CatchTime("search")
def search(smiles: str, finder: Finder, ans_count=None):
    fp = smiles_to_fingerprint(smiles)
    return list(finder.find(fp, ans_count))

In [14]:
class ComplexFinder(SearchEngine, Finder):   
    def find_all(self, fp):
        return self.search(fp)

In [15]:
class ThreadPoolFinder(ThreadPoolSearchEngine, Finder):   
    def find_all(self, fp):
        return self.search(fp)

In [16]:
# mcc100_2kk = ComplexFinder(data_path / '2kk_100_mcc')
# mcc100_2kk_async = AsyncFinder(data_path / '2kk_100_mcc')
# mcc100_17kk_heur2000 = ComplexFinder(data_path / '17kk_100_heur2000')
# mcc100_17kk_heur2000_async = AsyncFinder(data_path / '17kk_100_heur2000')
# mcc100_2kk_heur1000 = ComplexFinder(data_path / '2kk_100_heur1000')
# mcc100_2kk_heur1500 = ComplexFinder(data_path / '2kk_100_heur1500')
# mcc100_2kk_heur1700 = ComplexFinder(data_path / '2kk_100_heur1700')

In [17]:
# mcc100_17kk = ComplexFinder(data_path / '17kk_100_heur2000')

In [18]:
with open('../data/pubchem_994_queries.txt', 'r') as f:
    queries = list(map(lambda x: x.split()[0], f.read().strip().split('\n')))

In [19]:
len(queries), queries[:10]

(157,
 ['Cc1c[n]c2[nH]ccc2c1Cl',
  'OC(=O)c1c[n]c2[nH]ccc2c1Cl',
  'Cc1c(Cl)c[n]c2[nH]ccc21',
  'Oc1c[n]c2[nH]ccc2c1Cl',
  'N[C@@H](C[S@](=O)CCO)C(O)=O',
  'OC(=O)c1ccc2[nH]ccc2[n]1',
  'N[C@@H](C[S@@](=O)CCO)C(O)=O',
  'COC(=O)[C@@H](O)CC([O-])=O',
  'OC(=O)c1[n]ccc2[nH]ccc21',
  'CC[NH2+]CC(C)C'])

In [20]:
queries_fp = [smiles_to_fingerprint(smiles) for smiles in queries]

In [21]:
import random

In [22]:
queries_30 = random.sample(queries_fp, 30)

In [28]:
mcc100_2kk = ComplexFinder(data_path / '2kk_100_mcc')

0.015s -- ComplexFinder init time


In [29]:
%%time 
stat = FinderSpeedTester([mcc100_2kk]).test_all(queries_fp, verbose=True, ans_count=10)

-----
Test #000
2.428 -- ComplexFinder
-----
Test #001
1.014 -- ComplexFinder
-----
Test #002
1.301 -- ComplexFinder
-----
Test #003
1.232 -- ComplexFinder
-----
Test #004
3.919 -- ComplexFinder
-----
Test #005
1.588 -- ComplexFinder
-----
Test #006
2.426 -- ComplexFinder
-----
Test #007
1.493 -- ComplexFinder
-----
Test #008
1.136 -- ComplexFinder
-----
Test #009
0.042 -- ComplexFinder
-----
Test #010
1.433 -- ComplexFinder
-----
Test #011
0.006 -- ComplexFinder
-----
Test #012
0.484 -- ComplexFinder
-----
Test #013
1.474 -- ComplexFinder
-----
Test #014
0.051 -- ComplexFinder
-----
Test #015
0.004 -- ComplexFinder
-----
Test #016
0.198 -- ComplexFinder
-----
Test #017
2.563 -- ComplexFinder
-----
Test #018
1.139 -- ComplexFinder
-----
Test #019


KeyboardInterrupt: 

In [25]:
mcc100_17kk_heur2000_async = ThreadPoolFinder(data_path / '17kk_100_heur2000')

0.111s -- ThreadPoolFinder init time


In [26]:
%%time 
stat_async = FinderSpeedTester([mcc100_17kk_heur2000_async]).test_all(queries_30, verbose=True, ans_count=10)

-----
Test #000


AttributeError: 'BucketSearchEngine' object has no attribute 'full_fingerprints'

In [27]:
stat = stat_sync + stat_async

del executor


NameError: name 'stat_sync' is not defined

In [ ]:
stat

In [ ]:
stat.as_boxplot()

In [ ]:
stat.as_plot()

In [ ]:
stat.as_plot()

In [ ]:
from fp_utils.draw import MoleculeDrawer

In [ ]:
query = queries[0]
MoleculeDrawer.draw_one(query)

In [ ]:
answers = search(query, mcc100_17kk_heur2000_async, ans_count=10)

In [ ]:
len(answers)

In [ ]:
MoleculeDrawer.draw_many(answers, figsize=(20, 20), shape=(2, 3))

In [ ]:
answers = search(query, mcc100_17kk_heur2000, ans_count=10)
MoleculeDrawer.draw_many(answers, figsize=(20, 20), shape=(2, 3))